# Langevin bistable dimer with WCA potentials

Simple implementation of a bistable dimer in a reservoir with particles interacting through WCA potentials

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.visualization as msmrdvis
import msmrd2.tools.particleTools as particleTools
from msmrd2.integrators import langevinBAOAB
from msmrd2.potentials import WCA, pairBistable

In [ ]:
# Main parameters
numBathParticles = 500 #500 #500
numparticles = 2 + numBathParticles #Added distinguished particles (index 0 and 1)
# D = 3.0E-2 #1.0E-3 #(nm^2/ns) Note 1.0E-3 nm^2/ns = 1 micrometer^2/s #0.1
Gamma = 0.3 #30 # Friction coefficient (units of KbT/D = mass over time (gram/mol)/ns)
particlemass = 18.0 # (g/mol) approximately mass of water
distinguishedParticleMass = 3 * particlemass # (kg)
particleDiameter = 0.5 #0.3 # (nm) if 0.4nm spherical volume = water molecule volume approx
separationDistance = 1 * particleDiameter # minimum separation distance for initial condition
# For computations, we assume KbT=1, thus the force F must be: F=KbT f, where f is the force computed
# from the potential. This means the plotted potential is on reduced units (not the distances though);
KbT = 1

# Main parameters for integrator
dt = 0.05 #0.001 #0.0005 #0.05 # (ns)
seed = -1 # Seed = -1 used random device as seed
bodytype = 'point'

# Define simulation boundaries (choose either spherical or box)
boxsize = 15 #5 #(nm)
boundaryType = 'periodic'

# Parameters for WCA potential (rm=2^(1/6)sigma)
epsilon = 1.0
rm = particleDiameter # (diameter in nm)
sigma = rm * 2**(-1/6)
excludeParticleTypesPairs = [1] # Two particles of this same type wll not interact with each other.

# Parameters for pair bistable potential (will only act on distinguished particles (type 1))
x0 = 1.0*particleDiameter # location of first minima
rad = 1.0*particleDiameter # half the distance between minimas
distinguishedTypes = [1]
scalefactor = 2

# Over-damped Langevin integrator parameters
dt = 0.05
seed = -1 # Seed = -1 used random device as seed
bodytype = 'point'
Gamma = 0.3

In [ ]:
# Setup simulation

# Create list of particles that can be read from msmrd
partlist = particleTools.randomLangevinParticleListTwoDistinguished(numparticles, boxsize, separationDistance,
                                                        particlemass, seed, x0)
# Set distinguished particle (default type is zero)
partlist[0].setType(1)
partlist[1].setType(1)
partlist[0].setMass(distinguishedParticleMass)
partlist[1].setMass(distinguishedParticleMass)
    
# Define boundary
boxBoundary = msmrd2.box(boxsize, boxsize, boxsize, boundaryType)

# Define pair potential (potentialWCA will be set as aux potnetial because we want to store its data
# separately for our coarse-graining method).
potentialWCA = WCA(epsilon, sigma, excludeParticleTypesPairs)
potentialWCA.setForceCapValue(100.0)
potentialPairBistable = pairBistable(x0, rad, distinguishedTypes, scalefactor)

# Integrator definition
seed = 1
integrator = langevinBAOAB(dt, seed, bodytype, Gamma)
integrator.setBoundary(boxBoundary)
integrator.setPairPotential(potentialPairBistable)
integrator.setAuxPairPotential(potentialWCA)
integrator.setKbT(KbT)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
timesteps = 50000 #50000 #200000
stride = 5 #20 #1
datafile  = open('../../data/vmd/langevinDimerWCA.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            v0 = part.position
            if j == 0 or j == 1:
                datafile.write('type_1' + ' ' + ' '.join(map(str, v0)) + '\n')
            else:
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
    integrator.integrate(partlist)
    if i%1000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
print("Percentage complete: ", 100, " %")

In [ ]:
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_MoriZwanzig(numparticles = numparticles, outfname = "langevinDimerWCA", 
                              tclfname = "../../data/vmd/langevinDimerWCA_2vmd.tcl", 
                                 particleDiameter = 0.2)

To load the movie go to /data/vmd and run in a terminal "vmd -e langevinDimerWCA_2vmd.tcl".


## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate particle list and print only positions (prints positions and orientations of both particles)
timesteps = 100
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Positions", "Velocites"))
for i in range(timesteps):
    print('{:<10d}{:<10f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[0].velocity)))
    print('{:<10s}{:<10s}{:<40s}{:<50s}'.format(str(" "), str(" "), str(partlist[1].position), str(partlist[1].velocity)))
    integrator.integrate(partlist)